In [1]:
import yaml
import sys, os
sys.path.append(os.path.abspath("..")) 
from Model.BusinessGlossaryModel import BusinessGlossaryModel 
from Service.DataHubGlossaryImporter import DataHubGlossaryImporter
import Service.Utils as utils



full_path = "../template_business_glossary.yaml"

if full_path.endswith(".yaml") : 
    
    with open(full_path) as stream:
        data = yaml.safe_load(stream)
        dataset = BusinessGlossaryModel(**data)
        importer  = DataHubGlossaryImporter(dataset)


 

INFO:Service.Utils:Created node: urn:li:glossaryNode:operation
INFO:Service.Utils:Created node: urn:li:glossaryNode:operation.general
INFO:Service.Utils:Created term: urn:li:glossaryTerm:Driver3
INFO:Service.Utils:Upserted group urn:li:corpGroup:Operation_team
INFO:Service.Utils:Upserted group urn:li:corpGroup:BI_TEAM
INFO:Service.Utils:Owner urn:li:corpuser:BI_TEAM already exists, omitting write
INFO:Service.Utils:Owner urn:li:corpuser:Operation_team, type BUSINESS_OWNER added to dataset urn:li:glossaryTerm:Driver3


In [6]:
from datahub.emitter.mce_builder import (
    make_dataset_urn
)
from datahub.emitter.rest_emitter import DatahubRestEmitter
from datahub.metadata.schema_classes import (
    GlossaryNodeInfoClass,
    GlossaryTermInfoClass,
    GlossaryTermAssociationClass,
    GlossaryTermsClass,
    MetadataChangeProposalClass,
    OwnershipClass,
    OwnerClass,
    OwnershipTypeClass,
    SchemaFieldKey
    )
from datahub.emitter.mcp import MetadataChangeProposalWrapper

# === CONFIGURATION ===
DATAHUB_HOST = "http://localhost:8080"  # or your DataHub URL
DATASET_NAME = "127_0_0_1-3306.base_api.test15"  # e.g., 'fct_user_profile'
PLATFORM = "mysql"  # e.g., bigquery, snowflake, hive, etc.
ENV = "PROD"
COLUMN_NAME = "id"
GLOSSARY_TERM = "Driver3"  # Use full path: domain.termName

# === BUILD URNs ===
dataset_urn = make_dataset_urn(platform=PLATFORM, name=DATASET_NAME, env=ENV)
glossary_term_urn = f"urn:li:glossaryTerm:{GLOSSARY_TERM}" 

# === CREATE EMITTER ===
emitter = DatahubRestEmitter(gms_server=DATAHUB_HOST)

# === LINK COLUMN TO GLOSSARY TERM ===
mcp = MetadataChangeProposalWrapper(
    entityUrn=dataset_urn,
    entityType="dataset",
    aspectName="glossaryTerms",
    aspect=GlossaryTermsClass(
        terms=[
            GlossaryTermAssociationClass(
                urn=glossary_term_urn,
                context=SchemaFieldKey(fieldPath=COLUMN_NAME)  # field-level context
            )
        ]
    ),
    changeType=ChangeTypeClass.UPSERT,
)

# === EMIT THE PROPOSAL ===
emitter.emit(mcp)

print(f"Glossary term {GLOSSARY_TERM} linked to column {COLUMN_NAME} in dataset {DATASET_NAME}")


ImportError: cannot import name 'SchemaFieldKey' from 'datahub.metadata.schema_classes' (/home/mohsen1375/anaconda3/envs/datahub/lib/python3.12/site-packages/datahub/metadata/schema_classes.py)

In [ ]:
import time

from datahub.emitter.mce_builder import make_dataset_urn
from datahub.emitter.rest_emitter import DatahubRestEmitter
from datahub.emitter.mcp import MetadataChangeProposalWrapper
from datahub.metadata.schema_classes import (
    GlossaryTermAssociationClass,
    GlossaryTermsClass,
)
from datahub.metadata.com.linkedin.pegasus2avro.common import AuditStamp
 
DATAHUB_HOST = "http://localhost:8080"
DATASET_NAME = "127_0_0_1-3306.base_api.test15"
PLATFORM = "mysql"
ENV = "PROD"
COLUMN_NAME = "columnName1"
GLOSSARY_TERM = "Driver"
ACTOR_URN = "urn:li:corpuser:datahub"

dataset_urn = make_dataset_urn(platform=PLATFORM, name=DATASET_NAME, env=ENV)
glossary_term_urn = f"urn:li:glossaryTerm:{GLOSSARY_TERM}"

now_millis = int(time.time() * 1000)
audit_stamp = AuditStamp(time=now_millis, actor=ACTOR_URN)

glossary_terms_aspect = GlossaryTermsClass(
    auditStamp=audit_stamp,
    terms=[
        GlossaryTermAssociationClass(
            urn=glossary_term_urn,
            actor=ACTOR_URN,
            context= f'"fieldPath": "{COLUMN_NAME}"'
            
            ,  # use SchemaFieldKey
            attribution=None,
        )
    ],
)

mcp = MetadataChangeProposalWrapper(
    entityUrn=dataset_urn,
    aspect=glossary_terms_aspect,
    aspectName="glossaryTerms",
)

emitter = DatahubRestEmitter(gms_server=DATAHUB_HOST)
emitter.emit(mcp)

print(f"✅ Linked column '{COLUMN_NAME}' in dataset '{DATASET_NAME}' to glossary term '{GLOSSARY_TERM}'")


✅ Linked column 'columnName1' in dataset '127_0_0_1-3306.base_api.test15' to glossary term 'Driver'
